In [1]:
import pandas as pd
import numpy as np
#nem a sima helikopter nélküli model 1 kellene, hogy legyen a model 1 a végső eredménytáblákban?
# lehet úgy kellene, hogy metszet és az értékek, nem kivonva a metszet az adott értékből
# centering, mediánt kivonni nem lenne jobb? kiugró értékek kezelése miatt, pl van pár ami nagyon extra...
#EKB hiányzik mindenhol, de az EU országok sokszor benne vannak
#hitelbesorolás még jó lehet, az befolyásolja az IMF csomagot (vmi IV?)
# jegybanki függetlenség
# ZLB nem ZLB országok érdekes lehet, valami interakció?
#valami ellátási lánc mérésére szolgáló kontroll?
#central bank, IMF panel, non-standardized?
#változók jobb megértése
#negyedéves GDP kevés van
#számítások biztos mindenhol jók?
#mértékekegységek jók? importálásnál?
#országok tisztázása, egységesíteni ami ugyanaz
#kiugró értékek? ki kellene valahogy szűrni?
#energyimport miért viszi így el?
#kontrollok covid, worldbank is eurozónás aggregált kellene, hogy legyen?
# 3 éves reál GDP egyértelműen negatív a helicopternél, esetleg itt egy inflációs visszahatás lehet? inflációt emeli, és az visszahat a GDP-re is
#interakciós változó elemzése, biztos ez a legjobb? kettő metszete monetáris finanszírozás, ami marad az meg külön külön (viszont így sok a 0 érték...), hogy néz ki
# covid esetek első hullámig?
# financial corporation survey, oda a jegybank és az összes többi broad money intézmény benne van... ott is van claims on central governance - az nem lehet jó, hogy az összes claim-ből mennyi lett a jegybanknál, mennyi a "magánszektornál"?
# eu országoknál mit jelent a claims on central gov a jegybanki cuccban? az EKB claim-je? mi a különbség az euro area wide residency meg a sima között?
# az összeillesztésnél rosszak az ország nevek, így már pl. az nGDP-t nem tudja hozzá rendelni és NaN lesz, még előtte kell megjavítani...
# kormányzati változókat jobban megérteni? Melyik mit fed le? Ha az adósság nő, az megfogja a kormányzeti költekezést? Nem biztos, hogy csak adósságnöveléssel költekeznek? Az expenditure jobb lehet? Vagy az IMF? Ott viszont monetáris oldalról mi fogja meg jól?
# ha az expenditure meg a többi fiskális változó az general gov (?), akkor a claims-nek is annak kellene lennie? (central + local, társbizt benne van)

In [2]:
ISO_codes = pd.read_csv('/Users/kissr/Downloads/ISO_all.csv')
ISO_codes = ISO_codes[['name', 'alpha-3']]
ISO_codes['Country']=ISO_codes['name']
ISO_codes['ISO']=ISO_codes['alpha-3']
ISO_codes=ISO_codes[['ISO', 'Country']]

In [4]:
nGDP = pd.read_excel('/Users/kissr/Downloads/WEOOct2023all.xlsx', thousands=',', decimal='.')
nGDP = nGDP[['WEO Country Code', 'ISO', 'Country', 'WEO Subject Code', 2019, 2020]]
nGDP.rename(columns={'WEO Country Code': 'Country Code'}, inplace=True)
nGDP = nGDP[nGDP['WEO Subject Code']=='NGDP']
nGDP[[2019, 2020]] = nGDP[[2019, 2020]].apply(lambda x:pd.to_numeric(x, errors='coerce'))

In [4]:
#real GDP growth rate

GDP = pd.read_excel('/Users/kissr/Downloads/realGDPgrowth_IMF.xlsx', decimal='.')
GDP = GDP[['Real GDP growth (Annual percent change)', 2019, 2020, 2021, 2022, 2023]]
GDP.replace('no data', np.nan, inplace=True)
GDP[[2019, 2020, 2021, 2022, 2023]] = GDP[[2019, 2020, 2021, 2022, 2023]].apply(lambda x:pd.to_numeric(x, errors='coerce'))
GDP[[2019, 2020, 2021, 2022, 2023]]=GDP[[2019, 2020, 2021, 2022, 2023]]/100
GDP['rGDPgrowth_2019_control']=GDP[2019]*100
GDP['rGDPgrowth_2020_control']=GDP[2020]*100
GDP['rGDPgrowth_1y']=GDP[2021]*100
GDP['rGDP_growth_2y']=((1+GDP[2021])*(1+GDP[2022])-1)*100
GDP['rGDP_growth_3y']=((1+GDP[2021])*(1+GDP[2022])*(1+GDP[2023])-1)*100
GDP['GDP 2020']=GDP[2020]*100
GDP['GDP 2019-2021'] = ((1+GDP[2020])*(1+GDP[2021])-1)*100
GDP=GDP[['Real GDP growth (Annual percent change)','GDP 2019-2021', 'GDP 2020', 'rGDPgrowth_1y', 'rGDP_growth_2y', 'rGDP_growth_3y', 'rGDPgrowth_2019_control', 'rGDPgrowth_2020_control']]
GDP.rename(columns={'Real GDP growth (Annual percent change)': 'Country'}, inplace=True)

,Country,2019,2020,2021,2022,2023,rGDPgrowth_2019_control,rGDPgrowth_2020_control,rGDPgrowth_1y,rGDP_growth_2y,rGDP_growth_3y,GDP 2020,GDP 2019-2021
0,Afghanistan,0.039,-0.024,-0.207,NaN,NaN,3.9,-2.4,-20.7,NaN,NaN,-2.4,-22.6032
1,Albania,0.021,-0.033,0.089,0.048,0.036,2.1,-3.3,8.9,14.1272,18.235779,-3.3,5.3063
2,Algeria,0.010,-0.051,0.034,0.032,0.038,1.0,-5.1,3.4,6.7088,10.763734,-5.1,-1.8734
3,Andorra,0.020,-0.112,0.083,0.088,0.021,2.0,-11.2,8.3,17.8304,20.304838,-11.2,-3.8296
4,Angola,-0.007,-0.056,0.012,0.030,0.013,-0.7,-5.6,1.2,4.2360,5.591068,-5.6,-4.4672
...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,Other advanced economies,0.020,-0.016,0.057,0.026,0.018,2.0,-1.6,5.7,8.4482,10.400268,-1.6,4.0088
226,Sub-Saharan Africa,0.032,-0.016,0.047,0.040,0.033,3.2,-1.6,4.7,8.8880,12.481304,-1.6,3.0248
227,World,0.028,-0.028,0.063,0.035,0.030,2.8,-2.8,6.3,10.0205,13.321115,-2.8,3.3236
228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#negyedéves GDP növekedési üteme, sok országhoz hiányzik szóval nem biztos, hogy jó kontroll

qGDP = pd.read_excel('/Users/kissr/Downloads/IMF_qGDP.xlsx', sheet_name="qGDP", thousands=',', decimal='.')
qGDP.iloc[1:,1:]=qGDP.iloc[1:,1:].apply(lambda x:pd.to_numeric(x, errors='coerce'))
qGDP['negyedéves GDP kontroll']=(qGDP['2020Q2']/qGDP['2020Q1'])-(qGDP['2019Q2']/qGDP['2019Q1'])
qGDP['negyedéves GDP kontroll 2020Q1']=(qGDP['2020Q1']/qGDP['2019Q4'])-(qGDP['2019Q1']/qGDP['2018Q4'])
qGDP = qGDP[['Country', 'negyedéves GDP kontroll', 'negyedéves GDP kontroll 2020Q1' ]]
qGDP.to_excel('IMF_qGDP.xlsx', index=False)

In [14]:
#negyedéves infláció

qinflation = pd.read_excel('/Users/kissr/Downloads/IMF_qinflation.xlsx', sheet_name="qinflation", thousands=',', decimal='.')
qinflation.iloc[1:,1:]=qinflation.iloc[1:,1:].apply(lambda x:pd.to_numeric(x, errors='coerce'))
qinflation.replace('...', np.nan, inplace=True)
qinflation['negyedéves infláció kontroll']=(qinflation['2020Q2']/qinflation['2020Q1'])-(qinflation['2019Q2']/qinflation['2019Q1'])
qinflation = qinflation[['Country', 'negyedéves infláció kontroll']]
qinflation.to_excel('IMF_qinflation.xlsx', index=False)

In [20]:
# inflation

infl = pd.read_excel('/Users/kissr/Downloads/inflation_IMF.xlsx', decimal='.')
infl = infl[['Inflation rate, average consumer prices (Annual percent change)', 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]]
infl.replace('no data', np.nan, inplace=True)
infl[[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,2019, 2020, 2021, 2022, 2023]] = infl[[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,2019, 2020, 2021, 2022, 2023]].apply(lambda x:pd.to_numeric(x, errors='coerce'))
infl[[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,2019, 2020, 2021, 2022, 2023]]=infl[[2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,2019, 2020, 2021, 2022, 2023]]/100
infl['infl_1y']=infl[2021]*100
infl['infl_2y']=((1+infl[2021])*(1+infl[2022])-1)*100
infl['infl_3y']=((1+infl[2021])*(1+infl[2022])*(1+infl[2023])-1)*100
infl['infl_avg2009_2019']=(((1+infl[2010])*(1+infl[2011])*(1+infl[2012])*(1+infl[2013])*(1+infl[2014])*(1+infl[2015])*(1+infl[2016])*(1+infl[2017])*(1+infl[2018])*(1+infl[2019]))**(1/10)-1)*100
infl['infláció 2020']=infl[2020]*100
infl['infláció 2019-2021'] = ((1+infl[2020])*(1+infl[2021])-1)*100
infl=infl[['Inflation rate, average consumer prices (Annual percent change)', 'infláció 2020', 'infláció 2019-2021', 'infl_1y', 'infl_2y', 'infl_3y', 'infl_avg2009_2019']]
infl.rename(columns={'Inflation rate, average consumer prices (Annual percent change)':'Country'}, inplace=True)
infl.to_excel('infl_nov.xlsx', index=False)

In [ ]:
# DEPENDENT VARIABLES (+ avg inflation, + q GDP)

dep_vars = pd.merge(GDP, infl, on='Country', how='outer')
dep_vars = pd.merge(dep_vars, qGDP, on='Country', how='outer')
dep_vars

In [ ]:
# gross debt, nominal currency

grossdebt_level = pd.read_excel('/Users/kissr/Downloads/WEOOct2023all.xlsx', thousands=',', decimal='.')
grossdebt_level = grossdebt_level[['WEO Country Code', 'ISO', 'Country', 'WEO Subject Code', 2019, 2020, 2021, 2022, 2023]]
grossdebt_level.rename(columns={'WEO Country Code': 'Country Code'}, inplace=True)
grossdebt_level = grossdebt_level[grossdebt_level['WEO Subject Code']=='GGXWDG']
grossdebt_level[[2019, 2020]] = grossdebt_level[[2019, 2020]].apply(lambda x:pd.to_numeric(x, errors='coerce'))
grossdebt_level=pd.merge(grossdebt_level, nGDP[['Country Code', 2019, 2020]], on='Country Code', how='outer')
grossdebt_level['grossdebt_level_change']=((grossdebt_level['2020_x']-grossdebt_level['2019_x'])/grossdebt_level['2019_y'])*100
grossdebt_level = grossdebt_level[['Country Code', 'ISO', 'grossdebt_level_change']]
grossdebt_level

In [ ]:
# government expenditure, level, nominal currency

govexpenditure_level = pd.read_excel('/Users/kissr/Downloads/WEOOct2023all.xlsx', thousands=',', decimal='.')
govexpenditure_level = govexpenditure_level[['WEO Country Code','ISO', 'Country', 'WEO Subject Code', 2019, 2020, 2021, 2022, 2023]]
govexpenditure_level.rename(columns={'WEO Country Code': 'Country Code'}, inplace=True)
govexpenditure_level = govexpenditure_level[govexpenditure_level['WEO Subject Code']=='GGX']
govexpenditure_level[[2019, 2020]] = govexpenditure_level[[2019, 2020]].apply(lambda x:pd.to_numeric(x, errors='coerce'))
govexpenditure_level=pd.merge(govexpenditure_level, nGDP[['Country Code', 2019, 2020]], on='Country Code', how='outer')
govexpenditure_level['govexp_level_change']=((govexpenditure_level['2020_x']-govexpenditure_level['2019_x'])/govexpenditure_level['2019_y'])*100
govexpenditure_level = govexpenditure_level[['Country Code', 'ISO', 'govexp_level_change']]
govexpenditure_level

In [ ]:
# IMF fiscal monitor data, COVID-19 fiscal spending, % of 2020 GDP

IMFfiscal_pctGDP = pd.read_excel('/Users/kissr/Downloads/IMF_covid_fiscal.xlsx', sheet_name='imffiscal')
IMFfiscal_pctGDP =pd.merge(IMFfiscal_pctGDP, nGDP[['ISO', 'Country Code', 2019, 2020]], on="ISO", how='outer')
IMFfiscal_pctGDP['IMF_fiscalspending_ATL']=(IMFfiscal_pctGDP['Additional spending or foregone revenues']*IMFfiscal_pctGDP[2020])/IMFfiscal_pctGDP[2019]
IMFfiscal_pctGDP['IMF_fiscalspending_BTL']=(IMFfiscal_pctGDP['Liquidity support']*IMFfiscal_pctGDP[2020])/IMFfiscal_pctGDP[2019]
IMFfiscal_pctGDP['IMF_fiscalspending_total']=(IMFfiscal_pctGDP['Total_spending']*IMFfiscal_pctGDP[2020])/IMFfiscal_pctGDP[2019]
IMFfiscal_pctGDP['IMF_fiscalspending_ATL_pctTotal']=(IMFfiscal_pctGDP['IMF_fiscalspending_ATL']/IMFfiscal_pctGDP['IMF_fiscalspending_total'])*100
IMFfiscal_pctGDP=IMFfiscal_pctGDP[['Country', 'Country Code', 'ISO', 'IMF_fiscalspending_ATL', 'IMF_fiscalspending_BTL', 'IMF_fiscalspending_total', 'IMF_fiscalspending_ATL_pctTotal']]
IMFfiscal_pctGDP

In [ ]:
# FISCAL VARS, GOVERNMENT RELATED DATA (HELICOPTER: FISCAL)

fiscal_vars = pd.merge(grossdebt_level, govexpenditure_level, on='ISO', how='outer')
fiscal_vars = pd.merge(fiscal_vars,  IMFfiscal_pctGDP, on='ISO', how='outer')
fiscal_vars=fiscal_vars[['ISO', 'Country', 'Country Code', 'grossdebt_level_change', 'govexp_level_change', 'IMF_fiscalspending_ATL', 'IMF_fiscalspending_BTL', 'IMF_fiscalspending_total', 'IMF_fiscalspending_ATL_pctTotal']]
fiscal_vars

In [ ]:
# claims on central government, IMF panel database (just claims)

claimoncgov = pd.read_csv('/Users/kissr/Downloads/IMF_monetarystat_panel.csv')
claimoncgov = claimoncgov[['Country Name', 'Country Code', 'Time Period', 'Monetary, Central Bank Survey, Net Claims on Central Government, Claims on Central Government, Domestic Currency (FASAG_XDC)', 'Monetary, Central Bank Survey, Net Claims on Central Government, Claims on Central Government, Euros (FASAG_EUR)', 'Monetary, Central Bank Survey, Net Claims on Central Government, Claims on Central Government, US Dollars (FASAG_USD)']]
claimoncgov=claimoncgov.bfill(axis='columns')
claimoncgov['claimongov']=claimoncgov['Monetary, Central Bank Survey, Net Claims on Central Government, Claims on Central Government, Domestic Currency (FASAG_XDC)']
claimoncgov=claimoncgov[['Country Name', 'Country Code', 'Time Period', 'claimongov']]
claimoncgov = claimoncgov.pivot(index='Country Code', columns='Time Period', values='claimongov')
claimoncgov.reset_index(inplace=True)
claimoncgov = pd.merge(claimoncgov, nGDP, on='Country Code', how='outer')
claimoncgov['claimongov_change2020']=(claimoncgov['2020_x']-claimoncgov['2019_x'])/(claimoncgov['2019_y']*1000000000)*100
claimoncgov = claimoncgov[['Country Code', 'Country', 'ISO', 'claimongov_change2020']]
claimoncgov

In [ ]:
claimoncgov_gen = pd.read_csv('/Users/kissr/Downloads/IMF_monetarystat_panel.csv')
claimoncgov_gen = claimoncgov_gen[['Country Name', 'Country Code', 'Time Period', 'Monetary, Central Bank Survey, Claims on Other Sectors, Claims on State and Local Government, Domestic Currency (FASAOS_XDC)', 'Monetary, Central Bank Survey, Claims on Other Sectors, Claims on State and Local Government, Euros (FASAOS_EUR)', 'Monetary, Central Bank Survey, Claims on Other Sectors, Claims on State and Local Government, US Dollars (FASAOS_USD)']]
claimoncgov_gen=claimoncgov_gen.bfill(axis='columns')
claimoncgov_gen['claimongov_local']=claimoncgov_gen['Monetary, Central Bank Survey, Claims on Other Sectors, Claims on State and Local Government, Domestic Currency (FASAOS_XDC)']
claimoncgov_gen=claimoncgov_gen[['Country Name', 'Country Code', 'Time Period', 'claimongov_local']]
claimoncgov_gen = claimoncgov_gen.pivot(index='Country Code', columns='Time Period', values='claimongov_local')
claimoncgov_gen.reset_index(inplace=True)
claimoncgov_gen = pd.merge(claimoncgov_gen, nGDP, on='Country Code', how='outer')
claimoncgov_gen['claimongov_local_change2020']=(claimoncgov_gen['2020_x']-claimoncgov_gen['2019_x'])/(claimoncgov_gen['2019_y']*1000000000)*100
claimoncgov_gen = claimoncgov_gen[['Country Code', 'Country', 'ISO', 'claimongov_local_change2020']]
claimoncgov_gen['claimongov_gen_change2020'] = claimoncgov_gen['claimongov_local_change2020']+claimoncgov['claimongov_change2020']
claimoncgov_gen

In [ ]:
# cb total asset, IMF panel database

totalasset = pd.read_csv('/Users/kissr/Downloads/IMF_monetarystat_panel.csv')
totalasset = totalasset[['Country Name', 'Country Code', 'Time Period', 'Monetary, Central Bank, Assets, Domestic Currency (FAA_XDC)', 'Monetary, Central Bank, Assets, Euros (FAA_EUR)', 'Monetary, Central Bank, Assets, US Dollars (FAA_USD)']]
totalasset=totalasset.bfill(axis='columns')
totalasset['totalasset']=totalasset['Monetary, Central Bank, Assets, Domestic Currency (FAA_XDC)']
totalasset=totalasset[['Country Name', 'Country Code', 'Time Period', 'totalasset']]
totalasset = totalasset.pivot(index='Country Code', columns='Time Period', values='totalasset')
totalasset.reset_index(inplace=True)
totalasset.rename(columns={'Country Name': 'Country'}, inplace=True)
totalasset = pd.merge(totalasset, nGDP, on='Country Code', how='outer')
totalasset['totalasset_change2020']=(totalasset['2020_x']-totalasset['2019_x'])/(totalasset['2019_y']*1000000000)*100
totalasset = totalasset[['Country Code', 'Country', 'ISO', 'totalasset_change2020']]
totalasset

In [5]:
# monetary policy related short rate, IMF panel dataset

mprate = pd.read_csv('/Users/kissr/Downloads/IMF_monetarystat_panel.csv')
mprate = mprate[['Country Name', 'Country Code', 'Time Period', 'Financial, Interest Rates, Monetary Policy-Related Interest Rate, Percent per annum (FPOLM_PA)']]
mprate.rename(columns={'Country Name': 'Country'}, inplace=True)
mprate.rename(columns={'Financial, Interest Rates, Monetary Policy-Related Interest Rate, Percent per annum (FPOLM_PA)': 'monpolrate'}, inplace=True)
mprate = mprate.pivot(index='Country Code', columns='Time Period', values='monpolrate')
mprate.reset_index(inplace=True)
mprate['monpolrate_change_abs']=mprate[2020]-mprate[2019]
mprate['monpolrate_change_pct']=((mprate[2020]-mprate[2019])/mprate[2019])*100
mprate['ZLB'] = mprate[2019].apply(lambda x: np.nan if pd.isna(x) else 1 if x < 1 else 0)
mprate = pd.merge(mprate, nGDP, on='Country Code', how='outer')
mprate=mprate[['Country Code','Country', 'monpolrate_change_abs', 'monpolrate_change_pct', 'ZLB']]
mprate

C:\Users\kissr\AppData\Local\Temp\ipykernel_29628\4860505.py:12: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  mprate = pd.merge(mprate, nGDP, on='Country Code', how='outer')


,Country Code,Country,monpolrate_change_abs,monpolrate_change_pct,ZLB
0,111.0,United States,-1.5,-92.307692,0.0
1,112.0,United Kingdom,NaN,NaN,NaN
2,122.0,Austria,NaN,NaN,NaN
3,124.0,Belgium,NaN,NaN,NaN
4,128.0,Denmark,0.0,NaN,1.0
...,...,...,...,...,...
196,528,Taiwan Province of China,NaN,NaN,NaN
197,925,Turkmenistan,NaN,NaN,NaN
198,869,Tuvalu,NaN,NaN,NaN
199,299,Venezuela,NaN,NaN,NaN


In [ ]:
# government bond rate, IMF panel dataset (sok hiányzó adat)

longrate = pd.read_csv('/Users/kissr/Downloads/IMF_monetarystat_panel.csv')
longrate = longrate[['Country Name', 'Country Code', 'Time Period', 'Financial, Interest Rates, Government Securities, Government Bonds, Percent per annum (FIGB_PA)']]
longrate.rename(columns={'Country Name': 'Country'}, inplace=True)
longrate.rename(columns={'Financial, Interest Rates, Government Securities, Government Bonds, Percent per annum (FIGB_PA)': 'longrate'}, inplace=True)
longrate = longrate.pivot(index='Country Code', columns='Time Period', values='longrate')
longrate.reset_index(inplace=True)
longrate['longrate_change_abs']=longrate[2020]-longrate[2019]
longrate = pd.merge(longrate, nGDP, on='Country Code', how='outer')
longrate=longrate[['Country', 'Country Code', 'longrate_change_abs']]
longrate

In [ ]:
# CENTRAL BANK RELATED DATA (HELICOPTER: MONETARY)

cbdata = pd.merge(totalasset, claimoncgov_gen, on='Country Code', how='outer')
cbdata = pd.merge(cbdata, claimoncgov, on='Country Code', how='outer')
cbdata = pd.merge(cbdata, mprate, on='Country Code', how='outer')
cbdata = pd.merge(cbdata, longrate, on='Country Code', how='outer')
cbdata=cbdata[['Country Code', 'totalasset_change2020', 'claimongov_gen_change2020', 'claimongov_change2020', 'monpolrate_change_abs', 'monpolrate_change_pct', 'longrate_change_abs', 'ZLB']]
cbdata

In [ ]:
# CONTROLS (energy data is missing for 2020, last valid observation is used)

# World Bank data

worldbank=pd.read_csv('/Users/kissr/Downloads/worldbank_7a1c3888-c741-414d-80bc-0be3038ba28c_Data.csv')
worldbank = worldbank[(worldbank['Series Name']=='Total debt service (% of GNI)') | (worldbank['Series Name']=='Travel services (% of service exports, BoP)') | (worldbank['Series Name']=='Services, value added (% of GDP)') | (worldbank['Series Name']=='International tourism, receipts (% of total exports)') | (worldbank['Series Name']=='Government Effectiveness: Estimate') | (worldbank['Series Name']=='Energy imports, net (% of energy use)') | (worldbank['Series Name']=='Current health expenditure (% of GDP)')]
worldbank.replace('..', np.nan, inplace=True)
worldbank.iloc[:, 4:] = worldbank.iloc[:, 4:].ffill(axis='columns')
worldbank=worldbank[['Series Name', 'Country Name', 'Country Code', '2020 [YR2020]']]
worldbank = worldbank.pivot(index='Country Code', columns='Series Name', values='2020 [YR2020]')
worldbank.reset_index(inplace=True)
worldbank.rename(columns={'Country Code':'ISO'}, inplace=True)
worldbank.iloc[:, 1:] = worldbank.iloc[:, 1:].apply(lambda x:pd.to_numeric(x, errors='coerce'))
worldbank

In [ ]:
# CONTROLS

# Covid-19 (Our World in Data) - csak 2020 első félév

covid = pd.read_csv('/Users/kissr/Downloads/owid-covid-data.csv')
covid = covid[['iso_code', 'location', 'date', 'total_cases', 'total_deaths', 'stringency_index', 'population', 'median_age', 'aged_65_older', 'hospital_beds_per_thousand', 'life_expectancy']]
covid['date'] = pd.to_datetime(covid['date'])
covid['year'] = covid['date'].dt.year
covid = covid[covid['date']<'2020.07.01.']
covid = covid.pivot_table(values=['total_cases', 'total_deaths', 'stringency_index', 'population', 'median_age', 'aged_65_older', 'hospital_beds_per_thousand', 'life_expectancy' ], index='iso_code', aggfunc={'total_cases':"max", 'total_deaths': "max", 'stringency_index':"mean", 'population':"mean", 'median_age':"mean", 'aged_65_older':"mean", 'hospital_beds_per_thousand':"mean", 'life_expectancy':"mean"})
covid.reset_index(inplace=True)
covid.rename(columns={'iso_code':'ISO'}, inplace=True)
covid

In [ ]:
# INDEPENDENT VARIABLES

indep_vars = pd.merge(fiscal_vars, cbdata, on='Country Code', how='outer')
indep_vars

In [ ]:
# CONTROL VARIABLES

control_vars = pd.merge(covid, worldbank, on='ISO', how='outer')
control_vars

In [ ]:
eurozone = pd.read_excel('/Users/kissr/Downloads/eurozone.xlsx')
eurozone

In [ ]:
# FINAL HELICOPTER DB

helicopter = pd.merge(dep_vars, indep_vars, on='Country', how='outer')
helicopter = pd.merge(helicopter, control_vars, on='ISO', how='outer')
helicopter = pd.merge(helicopter, eurozone[['ISO', 'Eurozone']], on='ISO', how='outer')
helicopter['EZ_rGDPgrowth_2019_control'] = helicopter.apply(lambda row: np.nan if pd.isna(row['Eurozone']) else row['rGDPgrowth_2019_control'] if row['Eurozone'] == 'no' else 0.016, axis=1)
helicopter['EZ_rGDPgrowth_2020_control'] = helicopter.apply(lambda row: np.nan if pd.isna(row['Eurozone']) else row['rGDPgrowth_2020_control'] if row['Eurozone'] == 'no' else -0.061, axis=1)
helicopter['EZ_avginfl_control'] = helicopter.apply(lambda row: np.nan if pd.isna(row['Eurozone']) else row['infl_avg2009_2019'] if row['Eurozone'] == 'no' else 1.346545, axis=1)
helicopter

In [ ]:
# HELICOPTER VARIABLES

helicopter['heli_grossdebt_totalasset']=helicopter[['grossdebt_level_change', 'totalasset_change2020']].min(axis=1)
helicopter['grossdebt_grd_tota']=helicopter['grossdebt_level_change']-helicopter['heli_grossdebt_totalasset']
helicopter['totalasset_grd_tota']=helicopter['totalasset_change2020']-helicopter['heli_grossdebt_totalasset']

helicopter['heli_grossdebt_claims']=helicopter[['grossdebt_level_change', 'claimongov_change2020']].min(axis=1)
helicopter['grossdebt_grd_claims']=helicopter['grossdebt_level_change']-helicopter['heli_grossdebt_claims']
helicopter['claims_grd_claims']=helicopter['claimongov_change2020']-helicopter['heli_grossdebt_claims']

helicopter['heli_govexp_totalasset']=helicopter[['govexp_level_change', 'totalasset_change2020']].min(axis=1)
helicopter['govexp_govexp_tota']=helicopter['govexp_level_change']-helicopter['heli_govexp_totalasset']
helicopter['totalasset_govexp_tota']=helicopter['totalasset_change2020']-helicopter['heli_govexp_totalasset']

helicopter['heli_govexp_claims']=helicopter[['govexp_level_change', 'claimongov_change2020']].min(axis=1)
helicopter['govexp_govexp_claims']=helicopter['govexp_level_change']-helicopter['heli_govexp_claims']
helicopter['claims_govexp_claims']=helicopter['claimongov_change2020']-helicopter['heli_govexp_claims']

helicopter['heli_imf_totalasset']=helicopter[['IMF_fiscalspending_total', 'totalasset_change2020']].min(axis=1)
helicopter['imf_imf_tota']=helicopter['IMF_fiscalspending_total']-helicopter['heli_imf_totalasset']
helicopter['totalasset_imf_tota']=helicopter['totalasset_change2020']-helicopter['heli_imf_totalasset']

helicopter['heli_imf_claims']=helicopter[['IMF_fiscalspending_total', 'claimongov_change2020']].min(axis=1)
helicopter['imf_imf_claims']=helicopter['IMF_fiscalspending_total']-helicopter['heli_imf_claims']
helicopter['claims_imf_claims']=helicopter['claimongov_change2020']-helicopter['heli_imf_claims']


helicopter['heli (grossdebt claims on gen gov)']=helicopter[['grossdebt_level_change', 'claimongov_gen_change2020']].min(axis=1)
helicopter['grossdebt (grossdebt claims on gen gov)']=helicopter['grossdebt_level_change']-helicopter['heli (grossdebt claims on gen gov)']
helicopter['claims on gen gov (grossdebt claims on gen gov)']=helicopter['claimongov_gen_change2020']-helicopter['heli (grossdebt claims on gen gov)']

helicopter['heli (govexp claims on gen gov)']=helicopter[['govexp_level_change', 'claimongov_gen_change2020']].min(axis=1)
helicopter['govexp (govexp claims on gen gov)']=helicopter['govexp_level_change']-helicopter['heli (govexp claims on gen gov)']
helicopter['claims on gen gov (govexp claims on gen gov)']=helicopter['claimongov_gen_change2020']-helicopter['heli (govexp claims on gen gov)']

helicopter['heli (imf claims on gen gov)']=helicopter[['IMF_fiscalspending_total', 'claimongov_gen_change2020']].min(axis=1)
helicopter['imf (imf claims on gen gov)']=helicopter['IMF_fiscalspending_total']-helicopter['heli (imf claims on gen gov)']
helicopter['claims on gen gov (imf claims on gen gov)']=helicopter['claimongov_gen_change2020']-helicopter['heli (imf claims on gen gov)']